In [7]:
from prelu_cnn import CNN

from datasets import load_from_disk

# Load training dataset
dataset_path = "../processed_datasets/imagenet_processor"
ds = load_from_disk(dataset_path)
ds

Loading dataset from disk:   0%|          | 0/1550 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/61 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['pixel_values', 'labels'],
        num_rows: 1280967
    })
    validation: Dataset({
        features: ['pixel_values', 'labels'],
        num_rows: 50000
    })
})

In [8]:
ds = ds['validation'].select(range(1000))
# ds['pixel_values'].shape, type(ds['pixel_values'])
ds

Dataset({
    features: ['pixel_values', 'labels'],
    num_rows: 1000
})

In [9]:
inputs = ds['pixel_values']
labels = ds['labels']

In [10]:
import torch
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Move model and inputs to GPU if available
model = CNN(use_prelu=False, use_builtin_conv=True).to(device)

# Move tensors to the same device as the model
inputs = inputs.to(device)
labels = labels.to(device)

outputs = model(inputs)
outputs

Using device: cuda


tensor([[-3.7358e-02, -3.8836e-01,  4.9311e-01,  ...,  1.3499e-01,
         -1.3521e-01,  1.0928e-01],
        [-2.0279e-01, -1.7362e-01,  4.4675e-01,  ...,  1.7274e-01,
          1.5619e-01,  1.1215e-01],
        [-4.2935e-01, -3.1381e-01,  3.8551e-01,  ..., -1.5721e-01,
          6.4545e-02, -2.6540e-01],
        ...,
        [-7.5049e-02, -1.9266e-01,  2.4799e-01,  ..., -1.4077e-01,
          3.6100e-01,  2.8839e-01],
        [ 8.5665e-03, -1.0483e+00,  4.3870e-01,  ...,  3.2143e-01,
         -8.1018e-02, -4.6139e-01],
        [-2.2577e-01, -2.7216e-02,  3.4086e-05,  ...,  4.6153e-01,
          5.2840e-03,  1.1118e-01]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [11]:
import numpy as np

# axis=1 means we're finding the maximum value along the second dimension (columns)
# For a 2D tensor with shape (batch_size, num_classes), axis=1 gives us the class index
# with the highest probability for each sample in the batch
predictions = np.argmax(outputs.detach().cpu().numpy(), axis=1)
print("Predictions:", predictions)
print("Labels:", labels.cpu().numpy())
print("Correct predictions:", (predictions == labels.cpu().numpy()).sum(), "out of", len(predictions))


Predictions: [436  66 540 663 224  62 519 178 977 661 844 661 673 845   2 135 187 844
 467 160 661 920 309 135 788 414 979 540 763 902 489 492 937 330 481 892
 733 350 346 844 135 224 595 795 350 957 224 258 851 964 692 824 121 877
 937 661 846 661  21 224 341   2  82 179 679 643 410 920 957 135 121 845
 578 417 664 661 661 835 178 224 977   2 121 111 224 845 765 122 224 287
 844 661 567 906 801 224  88 285 224 108 224 788 307 128 679 844 135 818
  64 851 937 933 105 196 224 564 135 643 224 224 886 323 407 121  77 671
 181 595 481 514 980 343 240 844 671 876 135 887 941 135 178 661 200 636
 152 182 160 135  79 224 765  79 224 279 157 925 564 600 715 591 595 661
 224 105 643  42 851 661 648 633 224 844 349 844 964 771 661 567 410 489
 902 675 326 541 679   2 839  51 489  83 224 167  13 690 617 200 231 232
 600 771 763 844   2 679 654 578 844 330 664 788 844 673 285   2 643 920
 181 844 272  82 906 285 156 232 285 897 777 564 785 844 844 976 210 958
 924 224 224 135 661 418   2 224 414 9

In [12]:
import torch.nn as nn

loss_fct = nn.CrossEntropyLoss()
loss = loss_fct(outputs, labels)
loss

tensor(6.9679, device='cuda:0', grad_fn=<NllLossBackward0>)